<a href="https://colab.research.google.com/github/0ginny/TIL/blob/main/240409_Boltzmann_Machine_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### google drive mount

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Machine Learning A-Z (Codes and Datasets)/Boltzmann_Machines

/content/gdrive/MyDrive/Colab Notebooks/Machine Learning A-Z (Codes and Datasets)/Boltzmann_Machines


#Boltzmann Machine

실습 목표 : 사용자가 영화를 좋아할지 안좋아할지 (binary system)

[적용할 알고리즘에 대한 논문](https://drive.google.com/file/d/1S5AfopWtYzVNAGR0EdiNvBLQX-Q_2iQT/view?usp=drive_link)

##Downloading the dataset

###ML-100K

###ML-1M

##Importing the libraries

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn # neural network
import torch.nn.parallel # 병렬계산을 위해
import torch.optim as optim # optimizer
import torch.utils.data
from torch.autograd import Variable # 확률적 변위 하강을 위해

## Importing the dataset


In [5]:
movies = pd.read_csv('./ml-1m/movies.dat', sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1') # 특수문자도 인지하기 ㅜ이해

In [6]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
users = pd.read_csv('./ml-1m/users.dat', sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')

In [8]:
users #순번, 성별, 나이, 직업코드, 우편번호

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [9]:
ratings = pd.read_csv('./ml-1m/ratings.dat', sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')

In [10]:
ratings # 유저번호, 무비id, 평점(1~5), 타임스탬프

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


## Preparing the training set and the test set


현재 u1.base는 training set, u1.test는 test set이야

이렇게 구분되어있는 이유는 나중에 K-fold 교차검증을 하기 위해서.

In [11]:
training_set = pd.read_csv('./ml-100k/u1.base', delimiter = '\t') #tab 을 구분자로 쓸때는 sep 이 아니라 delimeter 인수를 써

In [12]:
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [13]:
training_set = np.array(training_set, dtype = 'int') # 딥러닝을 돌리기 위해 array 형식으로 변형

In [14]:
test_set = pd.read_csv('./ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


K-fold 교차검증을 위해 데이터를 분할 했지만,
각 데이터의 크기가 맞지 않을 수 있어.
아니면 user id가 맞지 않을 수(?) 있어.
그래서 최대 데이터의 id를 확인해볼 거야.

In [15]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])) )
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])) )

## Converting the data into an array with users in lines and movies in columns


사용자 행, 영와 열, 평점이 셀인 행렬을 만들거야.

In [16]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users +1):
        id_movies = data[:,1][data[:,0] == id_users ]
        id_ratings = data[:,2][data[:,0] == id_users ]
        ratings = np.zeros(nb_movies)
        ratings[id_movies -1 ] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [17]:
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [18]:
# torch tensor로 변형
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


좋은지 아닌지 처리를위해 binary 형태로 평점을 전환


RBM에서는 입력된 데이터로 입력되지 않은 데이터를 학습하므로, 입력형태와 출력 형태가 같아야해.

그래서 전처리부터 binary로 해주는 거야.

In [19]:
# 기존 0은 -1로 변환
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0 # 파이토치에선 or이 적용되지
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network


볼츠만 머신은 확률론적 그래픽 모델이야.

이번엔 모델클래스를 불러오는 것이 아닌 직접 만들어볼거야.

In [20]:
class RBM():
    #  초기화 visuable node, hidden node
    def __init__(self, nv, nh):
        #  최초 가중치는 랜덤하게 형성되어야해.
        # nh와 nv 의 크기를 가진 0~1까지 값을 가지는 torch weight tensor 생성
        self.w = torch.randn(nh, nv)
        # deeplearning을 위해선 2차원으로 바꿔줘야해.
        self.a = torch.randn(1, nh) # 배치, 편향
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.w.t())
        activation = wx + self.a.expand_as(wx) # 활성화 함수는 확률을 나타내
        p_h_given_v= torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.w)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h= torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    # contrast diversion
    def train(self, v0, vk,ph0, phk):
        self.w += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), dim = 0)
        self.a += torch.sum((ph0 - phk), dim = 0)

In [21]:
nv = len(training_set[0])
nh = 100 # 아무수나 해도 되니만 그래도 관련있는 수를 하자
batch_size = 100 # 아무숫자나 해도 괜찮긴해

In [22]:
rbm = RBM(nv, nh)

## Training the RBM


In [23]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    # real train
    # 배치에 있는 사용자 가져오기
    for id_user in range(0, nb_users - batch_size, batch_size):
        # 깁체인에 들어갈 -- 이며 매 반복마다 초기화 돼
        vk = training_set[id_user: id_user + batch_size]
        v0 = training_set[id_user: id_user + batch_size]
        # initial probability
        ph0 , _ = rbm.sample_h(v0)
        # k setp 대조발산
        # 깁체인 만들기
        for k in range(10):
            _, hk = rbm.sample_h(vk) # v0 는 바뀌지 않는 값, vk 는 업데이트 되는 값
            _, vk = rbm.sample_v(hk) # 가시적 노드의 업데이트
            # 평점이 없는 곳은 학습이 없게끔.
            vk[v0 < 0] = v0[v0<0]

        # train, phk가 필요해.
        phk,_ = rbm.sample_h(vk)

        # trian :: update weight
        rbm.train(v0, vk, ph0, phk)

        # 훈련 로스 없데이트
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))

        # 카운터 업데이트/ for normalize
        s += 1.

    # 훈련에 무슨일이 일어났는지 출력
    print(f'epoch : {str(epoch)} loss : {train_loss/s} ')

epoch : 1 loss : 0.3456915616989136 
epoch : 2 loss : 0.25167524814605713 
epoch : 3 loss : 0.25193169713020325 
epoch : 4 loss : 0.24807649850845337 
epoch : 5 loss : 0.24803321063518524 
epoch : 6 loss : 0.24713413417339325 
epoch : 7 loss : 0.2481215000152588 
epoch : 8 loss : 0.25004759430885315 
epoch : 9 loss : 0.24911843240261078 
epoch : 10 loss : 0.2485194206237793 


## Testing the RBM


In [27]:
# lose 는 1씩 늘려서 검사할거야.
test_loss = 0
s = 0.
for id_user in range(nb_users ):
    # 이건 그대로 유지해야해 / 그래야 훈련을 활성화 시킬 수 있어... 어렵다...
    v = training_set[id_user: id_user + 1]

    vt = test_set[id_user: id_user + 1]
    # 랜덤 워크를 한단계로 만들어야해.
    # 랜덤 워크는 확률이 같아야 하는거야.
    # 테스트는 블라인드 워크와 같아.
        # 확률이 같지 않을 수 있대, 그래서 한 단계로 나간것이 가장 나은 결과래
    if len(vt[vt >= 0])> 0 : # 평가해야할 항목이 있는가.
        _, h = rbm.sample_h(v) # v0 는 바뀌지 않는 값, vk 는 업데이트 되는 값
        _, v = rbm.sample_v(h) # 가시적 노드의 업데이트

        # 검사 로스 없데이트
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))

        # 카운터 업데이트/ for normalize
        s += 1.

# 훈련에 무슨일이 일어났는지 출력
print(f'loss : {test_loss/s} ')

loss : 0.2390177994966507 
